Copyright 2023 Province of British Columbia

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at 

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under he License.
the License.

## Imports and Parameters

In [1]:
# add our stuff to the path
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

# other stuff
import time
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)

# import our stuff
from importlib import reload
from src import connect, preprocess, synthetic, model

In [2]:
# PARAMETERS

# info to gain access to database, IDIR restricted 
CRED_PATH = '../credentials.txt'

# where model is stored. requires credentials.txt for full path 
MODEL_BASE_PATH = 'Model/Q22'

# which tables to access
RESPONSE_TABLE = 'dbo.AQ22ANCES'
CODE_TABLE = 'dbo.AQ22ANCES_Codes'
CLOSED_TABLE = 'dbo.Q22ANCESMultiResponse'
RESULTS_TABLE = 'dbo.AQ22ANCES_RESULTS'

# which column to use/create 
RESPONSE_COLUMN = 'aq22_cleaned'
OUTPUT_COLUMNS = 'q22ances_c'
N_COLUMNS = 5

# delimiter to send back with concatenated results
DELIMITER = 'μ' 

# amount of synthetic data to produce
N_PER_CATEGORY = 10_000
N_SYNTHETIC_MIXED = 200_000

# threshold for accepting as a flagged category
THRESHOLD = 0.5

# upper and lower thresholds for flagging as a possible category
TENTATIVE_UPPER = 0.75
TENTATIVE_LOWER = 0.25

## Read from Database

In [3]:
# Read in all data required to build model
connection = connect.create_connection(CRED_PATH)

# actual responses
df_open = connect.fetch_table(RESPONSE_TABLE, connection)

# codes to match
code_df = connect.fetch_table(CODE_TABLE, connection)

# closed respones (for multi response frequencies)
df_closed = connect.fetch_table(CLOSED_TABLE, connection)

C:\Users\lfredric\Work\dev\github_repos\bcgov\demo-nlp\notebooks\..\src\connect.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {table_name}', connection)


In [4]:
df_open.head()

,id,q22ances1,aq22ances1,q22ances_c01,q22ances_c02,q22ances_c03,q22ances_c04,q22ances_c05,q22ances2,aq22ances2,...,q22ances_c26,q22ances_c27,q22ances_c28,cycle,coding_comment,aq22ances1_cleaned,aq22ances2_cleaned,aq22ances3_cleaned,aq22ances4_cleaned,aq22ances5_cleaned
0,0039521,,,None,None,None,None,None,,,...,None,None,None,2,None,None,None,None,None,None
1,0039522,,,None,None,None,None,None,,,...,None,None,None,2,None,None,None,None,None,None
2,0039525,,,None,None,None,None,None,,,...,None,None,None,2,None,None,None,None,None,None
3,0039527,,,None,None,None,None,None,,,...,None,None,None,2,None,None,None,None,None,None
4,0039529,,,None,None,None,None,None,,,...,None,None,None,2,None,None,None,None,None,None


In [5]:
# Reshaping the dataframe
df_reshaped = preprocess.reshape_df(df_open)

In [6]:
# filter to only cycle 1 for now 
df_reshaped[df_reshaped.cycle == 1].head()

,id,cycle,q22ances,aq22ances,q22ances_c01,q22ances_c02,q22ances_c03,q22ances_c04,q22ances_c05,origin
1127,0000004,1,97,74% North Western European,42,None,None,None,None,4
7177,0000007,1,97,Germany,44,None,None,None,None,4
7238,0000011,1,24µ97,brazil,24,None,None,None,None,2
1445,0000013,1,97,"Scottland, England & Germany",42,44,None,None,None,4
7425,0000015,1,97,Kenya,14,None,None,None,None,1


## Preprocess Data

* Use only cycle 1 for training.
* Use the cleaned column.
* Correct spelling where possible (takes longer time to run).
* Create input and output tables for model (input table takes a while to produce)

In [7]:
# for training purposes, open responses should only be those from cycle 1
df = df_reshaped[df_reshaped.cycle == 1].reset_index(drop=True)
df.head()

,id,cycle,q22ances,aq22ances,q22ances_c01,q22ances_c02,q22ances_c03,q22ances_c04,q22ances_c05,origin
0,0000004,1,97,74% North Western European,42,None,None,None,None,4
1,0000007,1,97,Germany,44,None,None,None,None,4
2,0000011,1,24µ97,brazil,24,None,None,None,None,2
3,0000013,1,97,"Scottland, England & Germany",42,44,None,None,None,4
4,0000015,1,97,Kenya,14,None,None,None,None,1


In [8]:
# Use the cleaned column if available; otherwise, use the original column
df.loc[:, RESPONSE_COLUMN] = df.apply(
    lambda x: str(x.aq22ances).lower() if x.aq22ances is not None else np.nan, 
    axis=1
)

# Now only apply correct_spelling on non-null values
mask = df[RESPONSE_COLUMN].notnull()
df.loc[mask, RESPONSE_COLUMN] = df.loc[mask, RESPONSE_COLUMN].apply(preprocess.correct_spelling)

df.head()

,id,cycle,q22ances,aq22ances,q22ances_c01,q22ances_c02,q22ances_c03,q22ances_c04,q22ances_c05,origin,aq22_cleaned
0,0000004,1,97,74% North Western European,42,None,None,None,None,4,74% north western european
1,0000007,1,97,Germany,44,None,None,None,None,4,germany
2,0000011,1,24µ97,brazil,24,None,None,None,None,2,brazil
3,0000013,1,97,"Scottland, England & Germany",42,44,None,None,None,4,"scotland, england & germany"
4,0000015,1,97,Kenya,14,None,None,None,None,1,kenya


In [9]:
reload(preprocess)

<module 'src.preprocess' from 'C:\\Users\\lfredric\\Work\\dev\\github_repos\\bcgov\\demo-nlp\\notebooks\\..\\src\\preprocess.py'>

In [10]:
# get long form table of codes: step 1: turn countries into rows
code_df_long_tmp = preprocess.get_long_form_codes(code_df)
code_df_long_tmp.head(20)

,code,code_desc,description
0,88,I'm not able to be more specific,i'm not able to be more specific
1,99,Prefer not to answer,prefer not to answer
2,80000,Comment,comment
3,11,North Africa,north africa
4,11,North Africa,libyan arab jamahiriya
5,11,North Africa,sudan
6,11,North Africa,algeria
7,11,North Africa,western sahara
8,11,North Africa,tunisia
9,11,North Africa,morocco


In [12]:
# step 2: add additional rows for each country (nationalities, other spellings, etc.)
code_df_long = preprocess.get_long_form_codes_q22(code_df_long_tmp)

In [104]:
code_df_long.head()

,code,code_desc,description
0,80000,Comment,comment
1,11,North Africa,north africa
2,11,North Africa,sudan
3,11,North Africa,sudanese
4,11,North Africa,algeria


In [14]:
# build a training dataset for the model 
# first part of dataset: actual data
code_df_long['code'] = code_df_long['code'].astype(str)
df[RESPONSE_COLUMN] = df[RESPONSE_COLUMN].astype(str)

# INPUTS TO MODEL
headers = list(preprocess.get_scores('test', code_df_long, as_df = True).col_id.values)

input_df = preprocess.get_scores_from_df(df, RESPONSE_COLUMN, code_df_long, headers=headers)
input_columns = list(input_df.columns)
input_df = preprocess.convert_input(input_df)
display(input_df.head())

C:\Users\lfredric\Work\dev\github_repos\bcgov\demo-nlp\notebooks\..\src\preprocess.py:132: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df = response_df[response_column].apply(lambda x: get_scores(x, code_df_long, as_df=False))


,response,10,11,12,13,14,15,20,21,22,...,35,41,42,43,44,51,52,53,54,80
0,74% north western european,36.50,62.00,58.25,46.50,56.25,48.50,31.25,54.50,50.00,...,57.0,68.00,74.0,69.25,78.50,43.25,40.00,49.25,49.25,27.75
1,germany,44.25,69.50,69.50,51.75,53.50,44.50,23.75,62.00,53.75,...,71.0,62.00,69.0,57.00,100.00,43.50,46.50,56.25,53.50,29.00
2,brazil,24.00,48.25,51.25,50.00,44.50,59.25,34.75,44.50,52.25,...,64.5,49.00,40.0,57.00,34.50,50.00,33.00,52.50,38.75,0.00
3,"scotland, england & germany",39.25,52.25,59.75,62.00,62.75,49.75,30.00,58.75,58.50,...,68.0,60.00,100.0,58.25,89.25,58.75,53.50,47.50,58.00,26.25
4,kenya,34.75,45.50,61.75,45.00,100.00,47.00,28.75,51.50,61.75,...,62.5,42.25,55.0,49.50,48.50,47.50,42.25,40.00,55.00,35.75


In [107]:
preprocess.get_scores('test', code_df_long, as_df=False)

0       36
1       12
2       22
3       33
4       18
        ..
1955    22
1956    20
1957    40
1958     0
1959    35
Name: value, Length: 1960, dtype: int64

In [15]:
# OUTPUTS OF MODEL
# converts the coded columns into wide form 1/0 binary responses for every option 
output_df = preprocess.get_outputs_wide(df, RESPONSE_COLUMN, code_df_long, OUTPUT_COLUMNS, N_COLUMNS)
output_df.head()
output_columns = list(output_df.columns)

## Create Synthetic Data

* To augment our training data, produce synthetic data from the available phrases in the code list. 
* Both singular phrases and multi-response phrases are produced.
* The multi-response phrases are randomly generated according to weights associated with the non-written responses. 

In [16]:
reload(synthetic)

<module 'src.synthetic' from 'C:\\Users\\lfredric\\Work\\dev\\github_repos\\bcgov\\demo-nlp\\notebooks\\..\\src\\synthetic.py'>

In [17]:
# create synthetic data
# this section will create synthetic data that matches a single category based on available phrases 
extra_input_df, extra_output_df = synthetic.create_single_phrase_synthetic(
    output_df, 
    input_columns,
    output_columns,
    code_df_long,
    n_per_category = N_PER_CATEGORY,
    use_given = False
)

C:\Users\lfredric\Work\dev\github_repos\bcgov\demo-nlp\notebooks\..\src\preprocess.py:132: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df = response_df[response_column].apply(lambda x: get_scores(x, code_df_long, as_df=False))


01/26 -- Code: 42 -- Observations: 0 + 10000. Done.


C:\Users\lfredric\Work\dev\github_repos\bcgov\demo-nlp\notebooks\..\src\preprocess.py:132: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df = response_df[response_column].apply(lambda x: get_scores(x, code_df_long, as_df=False))


02/26 -- Code: 44 -- Observations: 0 + 10000. Done.


C:\Users\lfredric\Work\dev\github_repos\bcgov\demo-nlp\notebooks\..\src\preprocess.py:132: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df = response_df[response_column].apply(lambda x: get_scores(x, code_df_long, as_df=False))


...

C:\Users\lfredric\Work\dev\github_repos\bcgov\demo-nlp\notebooks\..\src\preprocess.py:132: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df = response_df[response_column].apply(lambda x: get_scores(x, code_df_long, as_df=False))


24/26 -- Code: 54 -- Observations: 0 + 10000. Done.


C:\Users\lfredric\Work\dev\github_repos\bcgov\demo-nlp\notebooks\..\src\preprocess.py:132: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df = response_df[response_column].apply(lambda x: get_scores(x, code_df_long, as_df=False))


25/26 -- Code: 202 -- Observations: 0 + 10000. Done.


C:\Users\lfredric\Work\dev\github_repos\bcgov\demo-nlp\notebooks\..\src\preprocess.py:132: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df = response_df[response_column].apply(lambda x: get_scores(x, code_df_long, as_df=False))


In [18]:
# create synthetic data
# this section will create synthetic data that matches multiple categories
mixed_input_df, mixed_output_df = synthetic.create_multi_phrase_synthetic(
    output_df,
    df_closed,
    input_columns,
    output_columns,
    code_df_long,
    N_SYNTHETIC_MIXED
)

200000/200000  |---------------------------------------------------------------------------------------------------->|
Creating synthetic data outputs... Done.
Creating synthetic data inputs... 

C:\Users\lfredric\Work\dev\github_repos\bcgov\demo-nlp\notebooks\..\src\preprocess.py:132: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df = response_df[response_column].apply(lambda x: get_scores(x, code_df_long, as_df=False))


Done.


In [19]:
mixed_input_df.head()

,response,80000_comment_ratio,11_north africa_ratio,11_sudan_ratio,11_sudanese_ratio,11_algeria_ratio,11_algerian_ratio,11_western sahara_ratio,11_sahrawi_ratio,11_tunisia_ratio,...,54_samoan_set,54_french polynesia_set,54_french polynesian_set,54_cook islander_set,54_cook islands_set,54_tonga_set,54_tongan_set,201_middle east_set,202_hawaii_set,100_indeterminate_set
0,jan mayen islands luxembourg,17,25,18,28,23,28,29,17,23,...,35,23,27,39,74,18,24,26,12,39
1,saint pierre bulgaria swedish western europe,8,25,16,23,24,27,31,24,20,...,16,23,26,32,29,8,12,22,16,28
2,jersey monegasque,17,21,18,40,25,24,32,17,17,...,35,42,41,20,21,36,35,36,9,40
3,russian federation belgian,18,32,26,29,30,35,30,30,30,...,25,38,42,31,26,26,31,27,25,41
4,bulgarian northern ireland netherlands,18,36,19,22,27,30,35,18,18,...,14,26,29,31,28,14,14,24,14,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,hungarian dutch,18,37,30,26,36,43,28,27,36,...,29,39,44,36,37,40,48,23,29,36
199996,miquelon sark liechtensteiner,22,29,18,27,22,27,33,22,28,...,23,36,35,29,29,24,23,35,17,43
199997,norwegian luxembourger,21,29,15,27,34,33,28,21,21,...,29,37,41,29,29,30,36,18,14,29
199998,tajikistan china indonesian sri lankan azerbaijan,14,26,15,21,21,25,32,21,21,...,15,34,36,26,26,11,15,20,18,26


In [20]:
chunk_size = 1000
chunks_extra_input = [extra_input_df.iloc[i:i+chunk_size] for i in range(0, len(extra_input_df), chunk_size)]
chunks_mixed_input = [mixed_input_df.iloc[i:i+chunk_size] for i in range(0, len(mixed_input_df), chunk_size)]

   response 80000_comment_ratio 11_north africa_ratio 11_sudan_ratio  \
0  svalbard                   0                    20             31   
1  scotland                  40                    30             46   
2   finland                  14                    32             33   
3   swedish                  14                    11             33   
4   denmark                  29                    32             33   

  11_sudanese_ratio 11_algeria_ratio 11_algerian_ratio  \
0                25               40                38   
1                38               27                38   
2                27               29                40   
3                40               29                27   
4                27               29                27   

  11_western sahara_ratio 11_sahrawi_ratio 11_tunisia_ratio  ...  \
0                      36               40               27  ...   
1                      27               27               27  ...   
2             

In [22]:
processed_chunks_extra_input = []
n_chunks = len(chunks_extra_input)
for idx, chunk in enumerate(chunks_extra_input):
    try:
        processed_chunk = preprocess.convert_input(chunk)
        processed_chunks_extra_input.append(processed_chunk)
        print(f"Chunk {idx+1:02}/{n_chunks} processed successfully for extra_input!", end = '\r')
    except Exception as e:
        print(f"Error in chunk {idx} for extra_input: {e}")

extra_input_df = pd.concat(processed_chunks_extra_input, ignore_index=True)

Chunk 0 processed successfully for extra_input!
Chunk 1 processed successfully for extra_input!
Chunk 2 processed successfully for extra_input!
Chunk 3 processed successfully for extra_input!
Chunk 4 processed successfully for extra_input!
Chunk 5 processed successfully for extra_input!
Chunk 6 processed successfully for extra_input!
Chunk 7 processed successfully for extra_input!
Chunk 8 processed successfully for extra_input!
Chunk 9 processed successfully for extra_input!
Chunk 10 processed successfully for extra_input!
Chunk 11 processed successfully for extra_input!
Chunk 12 processed successfully for extra_input!
Chunk 13 processed successfully for extra_input!
Chunk 14 processed successfully for extra_input!
Chunk 15 processed successfully for extra_input!
Chunk 16 processed successfully for extra_input!
Chunk 17 processed successfully for extra_input!
Chunk 18 processed successfully for extra_input!
Chunk 19 processed successfully for extra_input!
Chunk 20 processed successfull

In [26]:
processed_chunks_mixed_input = []
n_chunks = len(chunks_mixed_input)
for idx, chunk in enumerate(chunks_mixed_input):
    try:
        processed_chunk = preprocess.convert_input(chunk)
        processed_chunks_mixed_input.append(processed_chunk)
        print(f"Chunk {idx+1:02}/{n_chunks} processed successfully for mixed_input!", end='\r')
    except Exception as e:
        print(f"Error in chunk {idx} for mixed_input: {e}")

mixed_input_df = pd.concat(processed_chunks_mixed_input, ignore_index=True)

Chunk 0 processed successfully for mixed_input!
Chunk 1 processed successfully for mixed_input!
Chunk 2 processed successfully for mixed_input!
Chunk 3 processed successfully for mixed_input!
Chunk 4 processed successfully for mixed_input!
Chunk 5 processed successfully for mixed_input!
Chunk 6 processed successfully for mixed_input!
Chunk 7 processed successfully for mixed_input!
Chunk 8 processed successfully for mixed_input!
Chunk 9 processed successfully for mixed_input!
Chunk 10 processed successfully for mixed_input!
Chunk 11 processed successfully for mixed_input!
Chunk 12 processed successfully for mixed_input!
Chunk 13 processed successfully for mixed_input!
Chunk 14 processed successfully for mixed_input!
Chunk 15 processed successfully for mixed_input!
Chunk 16 processed successfully for mixed_input!
Chunk 17 processed successfully for mixed_input!
Chunk 18 processed successfully for mixed_input!
Chunk 19 processed successfully for mixed_input!
Chunk 20 processed successfull

In [27]:
# Concatenate data
final_input_df = pd.concat(
    [
        #input_df, 
        extra_input_df, 
        mixed_input_df
    ], 
    ignore_index=True).drop('response', axis=1).astype(int)
final_output_df = pd.concat(
    [
        #output_df, 
        extra_output_df, 
        mixed_output_df
    ]
    , 
    ignore_index=True).drop('response', axis=1).astype(int)

In [28]:
final_input_df.shape

(460000, 25)

In [29]:
final_output_df.shape

(460000, 26)

## Train Model

To deal with the fact we have many possible categorical outputs, use a simple random forest model that will handle multiple outputs better than other models.

* After training, save model. 

In [91]:
reload(model)

<module 'src.model' from 'C:\\Users\\lfredric\\Work\\dev\\github_repos\\bcgov\\demo-nlp\\notebooks\\..\\src\\model.py'>

In [31]:
n_estimators = 100
clf = model.create_model(final_input_df, final_output_df, n_estimators = n_estimators)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  3.8min


In [56]:
model.save_model(CRED_PATH, MODEL_BASE_PATH, clf, code_df_long)

Model already exists in directory //decimal.idir.bcgov/BCDemoSurveyCoding/Model/Q22. Overwrite? (y)/n:  y


In [ ]:
clf, code_df_long = model.load_model(CRED_PATH, MODEL_BASE_PATH)

## Produce Model Results

* Look at some results by hand to see if it makes sense
* Create results for entire hand-coded dataset
* Send results back to databse

In [103]:
sentence = 'chinese, english'
model.list_classes(sentence, code_df_long, clf, truncate_inputs=True, spellcheck=True)


TOP MATCHES FOR: chinese, english
CORRECTED TO:    chinese, english

97.00% (32)
East Asia

21.00% (42)
Northern Europe

20.00% (34)
South Asia

17.00% (44)
Western Europe

14.00% (41)
Eastern Europe

14.00% (35)
West Asia

12.00% (31)
Central Asia

12.00% (33)
Southeast Asia

7.00% (43)
Southern Europe

5.00% (54)
Polynesia



[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s


In [65]:
df_open.head()

,id,q22ances1,aq22ances1,q22ances_c01,q22ances_c02,q22ances_c03,q22ances_c04,q22ances_c05,q22ances2,aq22ances2,...,q22ances_c26,q22ances_c27,q22ances_c28,cycle,coding_comment,aq22ances1_cleaned,aq22ances2_cleaned,aq22ances3_cleaned,aq22ances4_cleaned,aq22ances5_cleaned
0,0039521,,,None,None,None,None,None,,,...,None,None,None,2,None,None,None,None,None,None
1,0039522,,,None,None,None,None,None,,,...,None,None,None,2,None,None,None,None,None,None
2,0039525,,,None,None,None,None,None,,,...,None,None,None,2,None,None,None,None,None,None
3,0039527,,,None,None,None,None,None,,,...,None,None,None,2,None,None,None,None,None,None
4,0039529,,,None,None,None,None,None,,,...,None,None,None,2,None,None,None,None,None,None


In [64]:
df.head()

,id,cycle,q22ances,aq22ances,q22ances_c01,q22ances_c02,q22ances_c03,q22ances_c04,q22ances_c05,origin,aq22_cleaned
0,0000004,1,97,74% North Western European,42,None,None,None,None,4,74% north western european
1,0000007,1,97,Germany,44,None,None,None,None,4,germany
2,0000011,1,24µ97,brazil,24,None,None,None,None,2,brazil
3,0000013,1,97,"Scottland, England & Germany",42,44,None,None,None,4,"scotland, england & germany"
4,0000015,1,97,Kenya,14,None,None,None,None,1,kenya


In [66]:
input_df.head()

,response,10,11,12,13,14,15,20,21,22,...,35,41,42,43,44,51,52,53,54,80
0,74% north western european,36.50,62.00,58.25,46.50,56.25,48.50,31.25,54.50,50.00,...,57.0,68.00,74.0,69.25,78.50,43.25,40.00,49.25,49.25,27.75
1,germany,44.25,69.50,69.50,51.75,53.50,44.50,23.75,62.00,53.75,...,71.0,62.00,69.0,57.00,100.00,43.50,46.50,56.25,53.50,29.00
2,brazil,24.00,48.25,51.25,50.00,44.50,59.25,34.75,44.50,52.25,...,64.5,49.00,40.0,57.00,34.50,50.00,33.00,52.50,38.75,0.00
3,"scotland, england & germany",39.25,52.25,59.75,62.00,62.75,49.75,30.00,58.75,58.50,...,68.0,60.00,100.0,58.25,89.25,58.75,53.50,47.50,58.00,26.25
4,kenya,34.75,45.50,61.75,45.00,100.00,47.00,28.75,51.50,61.75,...,62.5,42.25,55.0,49.50,48.50,47.50,42.25,40.00,55.00,35.75


In [95]:
reload(model)

<module 'src.model' from 'C:\\Users\\lfredric\\Work\\dev\\github_repos\\bcgov\\demo-nlp\\notebooks\\..\\src\\model.py'>

In [96]:
results_df = model.produce_results(
    df, input_df, output_df, 
    clf,
    OUTPUT_COLUMNS,
    N_COLUMNS,
    question = 'Q22',
    threshold=THRESHOLD,
    tentative_lower = TENTATIVE_LOWER,
    tentative_upper = TENTATIVE_UPPER,
    delimiter = DELIMITER
)

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.6s


In [97]:
results_df.head()

,id,cycle,q22ances,aq22ances,origin,aq22_cleaned,q22ances_c01,q22ances_c02,q22ances_c03,q22ances_c04,q22ances_c05,match,original_matched,extra_categories,n_original_categories,n_model_categories,tentative_categories,outside_continent,inside_continent
0,0000004,1,97,74% North Western European,4,74% north western european,42,44,None,None,None,0,1.0,1,1,2,42,0,1
1,0000007,1,97,Germany,4,germany,44,None,None,None,None,1,1.0,0,1,1,None,0,1
2,0000011,1,24µ97,brazil,2,brazil,24,None,None,None,None,1,1.0,0,1,1,None,0,1
3,0000013,1,97,"Scottland, England & Germany",4,"scotland, england & germany",42,44,None,None,None,1,1.0,0,2,2,None,0,1
4,0000015,1,97,Kenya,1,kenya,14,None,None,None,None,1,1.0,0,1,1,None,0,1


In [ ]:
# save back to database
engine = connect.create_connection(CRED_PATH, sqlalchemy=True)

# for initial save of cycle 1, always replace. any subsequent inputs should be appended
connect.save_table(results_df, RESULTS_TABLE, engine, how='replace')